In [1]:
from func import *

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix

from lightfm import LightFM, cross_validation
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.data import Dataset

from sklearn.model_selection import train_test_split

from enum import Enum, auto

import pickle

/Users/janpetr/Desktop/JobHunt/_PythonProjects/DatasenticsTask/func.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('data/Books.csv')
/Users/janpetr/anaconda3/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
class PredictionType(Enum):
    WORST = auto()
    BEST = auto()

<h1>Obsah:</h1>

<ul>
    
- Čtení dat, preprocessing, příprava vstupů pro model
- Trénování modelu
- Evaluace modelu
- Predikce pro náhodného uživatele z existujícího datasetu
- Zavedení nového uživatele (hodnotil knihy, které neměly hodnocení)
- Zavedení nového uživatele (hodnotil knihy, které již byly hodnocené)
- (Cold start problem) Zavedení nového uživatele - bez hodnocení, pouze na základě user features
</ul>

<h1>Poznámky</h1>

In [3]:
# udělat content based filtering nejdříve

In [4]:
# nový uživatel bez hodnocení -> ?
# nová kniha bez hodnocení -> ?

In [5]:
# in production - nejdříve content base filtering pro itemy, které ještě nebyly hodnocené - získat hodnocení (cold start)
# teprve poté collaborative filtering

In [6]:
# interpretace výsledků https://github.com/lyst/lightfm/issues/155

In [7]:
# Proč LightFM - hybrid content info a ratings + explicit i implicit feedback

In [8]:
# nápad - chatgpt api na zjištění více o knihách - žánr, počet stran atd.

<h1>Funkce</h1>

In [9]:
def interactions_filtering(dataset):
    """
    Pouze první trénovací dataset. Nové interakce neočisťujeme.
    """
    
    # Definovat společné IDs s Books i Users
    common_ids_isbn = pd.merge(ratings, books, on="ISBN", how="inner")["ISBN"].unique()
    common_ids_userid = pd.merge(ratings, users, on="User-ID", how="inner")["User-ID"].unique()

    # Očistit dataset o hodnoty, které nejsou v Book a Users - nutné
    dataset = dataset[dataset["ISBN"].isin(common_ids_isbn) & dataset["User-ID"].isin(common_ids_userid)]
    
    # Pouze explicitní feedback, nikoli implicitní (viz zadání)
    dataset = dataset[dataset["Book-Rating"] > 0] # pozitivní vliv na metriky
    
    # Pouze knihy, které byly hodnocené xkrát a více
    dataset = dataset.groupby("ISBN").filter(lambda x: len(x)>=25)
    
    # Pouze uživatelé, kteří hodnotily x a více knih (cold start)
    dataset = dataset[dataset.groupby("User-ID").ISBN.transform('nunique')>=10]
    
    # Upravit velikost datasetu kvůli délce trénování a evaluace modelu.
    if len(dataset) > 100000:
        dataset = dataset.sample(100000)
    
    return dataset

def dataset_preprocessing(dataset):
    # Ponechat hodnocení v původním stavu
    dataset["OriginalRating"] = dataset["Book-Rating"]
    
    # Každý má hodnotící standard nastavený jinak. Mirka Spáčilová dává nejvíce 7/10.
    #dataset["Book-Rating"] = dataset.groupby("User-ID")["Book-Rating"].transform(lambda x: x - x.mean())
    
    # !Není možné použít v kombinaci s horní metodou. Exponenciálně větší šance na doporučení s přibývajícím hodnocením.
    #dataset["Book-Rating"] = dataset["Book-Rating"].transform(lambda x: x*x)
    
    return dataset

def split_location(location):
    parts = location.split(",")

    while len(parts) < 3:
        parts.insert(0, "")  
    city = parts[-3].strip() if parts[-1] else "" 
    region = parts[-2].strip() if len(parts) > 1 and parts[-2] else ""
    country = parts[-1].strip() if len(parts) > 2 and parts[-3] else ""
    
    return pd.Series([city, region, country], index=['city', 'region', 'country'])

def input_preparation(dataframe, dataset):
    #dataset_tuple = [tuple(row) for row in dataframe.itertuples(index=False)]
    dataset_tuple = list(zip(dataframe["User-ID"], dataframe["ISBN"], dataframe["Book-Rating"]))
    interactions, weights = dataset.build_interactions(i for i in dataset_tuple)
    return interactions, weights

def check_isbns(isbn_list):
    isbn_set = set(isbn_list)
    books_isbn_set = set(books["ISBN"])
    
    common_isbns = isbn_set.intersection(books_isbn_set)
    
    invalid_isbns = [x for x in isbn_set if x not in books_isbn_set]
    
    print(f"{len(common_isbns)}/{len(isbn_set)} valid ISBNs.")
    
    if len(common_isbns) != len(isbn_set):
        print("")
        print ("Invalid ISBNs:")
        for isbn in invalid_isbns:
            print(isbn)

def previous_ratings(userid, noitems=None):
    user_ratings = ratings[ratings["User-ID"]==userid]
    user_books = pd.merge(user_ratings, books, on="ISBN")
    user_books = user_books.sort_values(by="Book-Rating", ascending=False)
    
    if noitems is not None:
        user_books = user_books[:noitems]

    print(f"User ID: {userid}. Previous ratings (highest to lowest):")

    for _, row in user_books.iterrows():
        print(f"- {int((row['OriginalRating']))}*, {row['Book-Title']}, {row['Book-Author']}, {row['Year-Of-Publication']} (ISBN: {row['ISBN']})")
        
def prediction(userid, where, noitems, remove_rated):
    usercode = user_id_map.get(userid)
    
    _, n_items = train_interactions.shape
    scores = pd.Series(model.predict(usercode,np.arange(n_items)))

    items = pd.DataFrame(scores)

    items.index = items.index.map(item_id_inverse_map)
    items = items.reset_index(names="ISBN").rename(columns={0: "prob"})

    if remove_rated == True:
        ratedisbn = ratings[ratings["User-ID"]==userid]["ISBN"]
    elif remove_rated == False:
        ratedisbn = []

    booksfinal = pd.merge(books,items,on="ISBN",how="inner")

    if where == PredictionType.BEST:
        booksfinal = booksfinal.sort_values(by="prob", ascending=False)
        print(f"Predicted: best {noitems}:")
    elif where == PredictionType.WORST:
        booksfinal = booksfinal.sort_values(by="prob", ascending=True)
        print(f"Predictd: worst {noitems}:")

    for _, row in booksfinal[~booksfinal["ISBN"].isin(ratedisbn)][:noitems].iterrows():
        print(f"- {row['Book-Title']}, {row['Book-Author']}, {row['Year-Of-Publication']} (ISBN: {row['ISBN']})")

In [48]:
train_interactions.shape

(1515, 1490)

<h1>Data Reading</h1>

In [10]:
ratings = pd.read_csv("data/Ratings.csv")
books=pd.read_csv('data/Books.csv')
users=pd.read_csv("data/Users.csv")

/var/folders/g4/1x4xl1_92cj02tdf3gqfw5qm0000gn/T/ipykernel_12832/2216795062.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('data/Books.csv')


<h1>DataFrame Preprocessing</h1>

<h2>Ratings</h2>

In [11]:
ratings = interactions_filtering(ratings)
ratings = dataset_preprocessing(ratings)
ratings

,User-ID,ISBN,Book-Rating,OriginalRating
1456,277427,002542730X,10,10
1474,277427,0061009059,9,9
1522,277427,0316776963,8,8
1543,277427,0345413903,10,10
1578,277427,0385424736,9,9
...,...,...,...,...
1149551,276680,0440221595,8,8
1149564,276680,0452283205,7,7
1149592,276680,0743203631,7,7
1149604,276680,0743486226,6,6


In [12]:
interactions_unique_users = ratings["User-ID"].unique()
interactions_unique_isbns = ratings["ISBN"].unique()

print("Unique users: ", len(interactions_unique_users))
print("Unique books: ", len(interactions_unique_isbns))

Unique users:  1515
Unique books:  1490


<h2>Users (features)</h2>

In [13]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [14]:
# Rozdělení Location feature na City, Region a Country
users[["City", "Region", "Country"]] = users["Location"].apply(lambda x: split_location(x))

# Nahrazení NaN hodnot ve věku nejčetnější hodnotou v datasetu
users["Age"] = users["Age"].fillna(users["Age"].mode()[0])

# Odstranění uživatelů, kteří nejsou v Ratings datasetu - kvůli dimenzionalitě
users = users[users["User-ID"].isin(set(interactions_unique_users))]

# List všech features - vstup pro dataset
unique_user_features = users[["Age", "City", "Region", "Country"]].values.flatten().tolist()

# Veškeré features spojené s User Id
user_features_tpl = [(user_id, [age, city, region, country]) 
             for user_id, age, city, region, country 
             in zip(users["User-ID"], users["Age"], users["City"], users["Region"], users["Country"])]

<h2>Books (features)</h2>

In [15]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [16]:
# Odstranění knih, kteří nejsou v Ratings datasetu - kvůli dimenzionalitě
books = books[books["ISBN"].isin(set(interactions_unique_isbns))]

# List všech features - vstup pro dataset
unique_book_features = books[["Book-Author", "Year-Of-Publication", "Publisher"]].values.flatten().tolist()

# Veškeré features spojené s ISBN
book_features_tpl = [(isbn, [author, year, publisher]) 
             for isbn, author, year, publisher 
             in zip(books["ISBN"], books["Book-Author"], books["Year-Of-Publication"], books["Publisher"])]

<h1>Train/Test split (Interactions)</h1>

In [17]:
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

<h1>Input preparation</h1>

<h2>Interactions</h2>

In [18]:
full_dataset = Dataset()
full_dataset.fit(users=interactions_unique_users, 
                 items=interactions_unique_isbns, 
                 user_features=unique_user_features,
                 item_features=unique_book_features)


user_id_map, user_features_map, item_id_map, item_features_map = full_dataset.mapping()

user_id_inverse_map = {v: k for k, v in user_id_map.items()}
item_id_inverse_map = {v: k for k, v in item_id_map.items()}

In [19]:
train_interactions, train_weights = input_preparation(train, full_dataset)
test_interactions, test_weights = input_preparation(test, full_dataset)

<h2>Features</h2>

In [20]:
user_features_matrix = full_dataset.build_user_features(user_features_tpl, normalize=False)

In [21]:
item_features_matrix = full_dataset.build_item_features(book_features_tpl, normalize=False)

<h1>Model</h1>

In [22]:
model = LightFM(loss='warp',
                learning_schedule="adagrad",
                no_components = 30,
                k=15,
               )

In [23]:
model.fit(train_interactions,
          sample_weight=train_weights,
          user_features=user_features_matrix,
          item_features=item_features_matrix,
          epochs=50, 
          num_threads=4,
          verbose=True,
         )

Epoch: 100%|████████████████████████████████████| 50/50 [00:03<00:00, 15.89it/s]


<h1>Evaluation</h1>

In [24]:
train_auc = auc_score(model,
                      train_interactions,
                      user_features=user_features_matrix,
                      item_features=item_features_matrix,
                     ).mean()

test_auc = auc_score(model,
                     test_interactions,
                     train_interactions=train_interactions,
                     user_features=user_features_matrix,
                     item_features=item_features_matrix,
                    ).mean()

print(f"Train AUC: {train_auc}")
print(f"Test AUC: {test_auc}")

Train AUC: 0.9813355803489685
Test AUC: 0.73013836145401


In [25]:
train_precision = precision_at_k(model,
                                 train_interactions,
                                 user_features=user_features_matrix,
                                 item_features=item_features_matrix,
                                 k=10,
                                ).mean()

test_precision = precision_at_k(model,
                                test_interactions,
                                train_interactions=train_interactions,
                                user_features=user_features_matrix,
                                item_features=item_features_matrix,
                                k=10,
                                ).mean()

print(f"Train Precision: {train_precision}")
print(f"Test Precision: {test_precision}")

Train Precision: 0.3851485252380371
Test Precision: 0.02751540020108223


In [26]:
train_recall = recall_at_k(model,
                           train_interactions,
                           user_features=user_features_matrix,
                           item_features=item_features_matrix,
                           k=10,
                          ).mean()

test_recall = recall_at_k(model,
                          test_interactions,
                          train_interactions=train_interactions,
                          user_features=user_features_matrix,
                          item_features=item_features_matrix,
                          k=10,
                         ).mean()

print(f"Train Recall: {train_recall}")
print(f"Test Recall: {test_recall}")

Train Recall: 0.2847966615452277
Test Recall: 0.07063173729723672


<h1>Single user Prediction</h1>

In [27]:
userid = int(ratings.sample(1)["User-ID"]) # random user
userid

/var/folders/g4/1x4xl1_92cj02tdf3gqfw5qm0000gn/T/ipykernel_12832/3419819421.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  userid = int(ratings.sample(1)["User-ID"]) # random user


67840

In [28]:
previous_ratings(userid, 10)

User ID: 67840. Previous ratings (highest to lowest):
- 10*, Full House (Janet Evanovich's Full Series), Janet Evanovich, 2002 (ISBN: 0312983271)
- 10*, Four Blind Mice, James Patterson, 2003 (ISBN: 0446613266)
- 10*, Fatal Voyage, Kathy Reichs, 2002 (ISBN: 0671028375)
- 10*, Misery, Stephen King, 1990 (ISBN: 0670813648)
- 10*, The Lord of the Rings (Movie Art Cover), J.R.R. Tolkien, 2001 (ISBN: 0618129022)
- 10*, God Save the Sweet Potato Queens, JILL CONNER BROWNE, 2001 (ISBN: 060980619X)
- 10*, The Sweet Potato Queens' Book of Love, JILL CONNER BROWNE, 1999 (ISBN: 0609804138)
- 10*, Sleep No More, Greg Iles, 2003 (ISBN: 0451208765)
- 10*, Cold Mountain : A Novel, CHARLES FRAZIER, 1998 (ISBN: 0375700757)
- 10*, Seabiscuit: An American Legend, LAURA HILLENBRAND, 2002 (ISBN: 0449005615)


In [29]:
prediction(userid, PredictionType.BEST, 10, remove_rated=False)

Predicted: best 10:
- The Pilot's Wife, Anita Shreve, 2001 (ISBN: 0316788228)
- The Girl Who Loved Tom Gordon : A Novel, Stephen King, 1999 (ISBN: 0684867621)
- 3rd Degree, James Patterson, 2004 (ISBN: 0316603570)
- Moonlight Becomes You, Mary Higgins Clark, 1997 (ISBN: 0671867113)
- Saint Maybe, ANNE TYLER, 1992 (ISBN: 0804108749)
- Life,the Universe and Everything, Douglas Adams, 0 (ISBN: 0330267388)
- SHIPPING NEWS, Annie Proulx, 1994 (ISBN: 0671510053)
- The Concrete Blonde (A Harry Bosch Novel), Michael Connelly, 1995 (ISBN: 0312955006)
- Prey, Michael Crichton, 2003 (ISBN: 0061015725)
- Black Notice, Patricia Daniels Cornwell, 1999 (ISBN: 0399145087)


<h1>New user: books unrated before</h1>

<h2>Update datasetu a modelu</h2>

In [30]:
"""
New User Testing
user_id = 123456

Sci-Fi Fantasy fan
Crime not fan

375704027 -> 9 (Murakami)
385177259 -> 10 (Asimov)
345309014 -> 10 (Asimov)
60929871 -> 8 (Brave New World, Huxley)
553106635 -> 10 (Game of Thrones)
3548245978 -> 1 (Nesbo)
3548255442 -> 2 (Nesbo)
877959943 -> 1 (McBain)
877959870 -> 4 (McBain)
446517380 -> 3 (McBain)
553256785 -> 2 (A. Christie)
671555235 -> 3 (A. Christie)
"""

ratings_new_userid = 123456
ratings_new_isbn = ["0375704027", 
                    "0385177259", 
                    "0345309014", 
                    "0060929871", 
                    "0553106635", 
                    "3548245978", 
                    "3548255442", 
                    "0877959943", 
                    "0877959870", 
                    "0446517380", 
                    "0553256785", 
                    "0671555235"]
ratings_new_rating = [9, 10, 10, 8, 10, 1, 2, 1, 4, 3, 2, 3]
ratings_new = pd.DataFrame({
    "User-ID": ratings_new_userid,
    "ISBN": ratings_new_isbn,
    "Book-Rating": ratings_new_rating
})

In [31]:
check_isbns(ratings_new_isbn)

1/12 valid ISBNs.

Invalid ISBNs:
0553106635
0671555235
0345309014
0375704027
0553256785
0385177259
0877959943
3548245978
0446517380
3548255442
0877959870


In [32]:
# Preprocessing nových dat
ratings_new = dataset_preprocessing(ratings_new)

# Spojení v celým Ratings datasetem
ratings = pd.concat([ratings, ratings_new], ignore_index=True)

In [33]:
full_dataset.fit_partial(users=ratings_new["User-ID"].unique(), 
                         items=ratings_new["ISBN"].unique())

user_id_map, user_feature_map, item_id_map, item_feature_map = full_dataset.mapping()

user_id_inverse_map = {v: k for k, v in user_id_map.items()}
item_id_inverse_map = {v: k for k, v in item_id_map.items()}

In [34]:
new_interactions, new_weights = input_preparation(ratings_new, full_dataset)

In [35]:
# Fit Partial for Model (model.fit_partial) nefunguje pro nové uživatele - dimenzionalita nesedí

model.fit(new_interactions,
          sample_weight=new_weights,
          #user_features=user_features_matrix,
          #item_features=item_features_matrix,
          num_threads=4,
          verbose=True,
         )

Epoch: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 2106.63it/s]


<h2>Predikce</h2>

In [36]:
# Problém je, že knihy v novém datasetu nebyly hodnoceny vícekrát, takže predikce pokulhává.

user_id = 123456
no_items = 10

previous_ratings(user_id, no_items) 
print("")
prediction(user_id, PredictionType.BEST, no_items, remove_rated=False)

User ID: 123456. Previous ratings (highest to lowest):
- 8*, Brave New World, Aldous Huxley, 1998 (ISBN: 0060929871)

Predicted: best 10:
- Brave New World, Aldous Huxley, 1998 (ISBN: 0060929871)
- Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1), Tim Lahaye, 1996 (ISBN: 0842329129)
- A Second Chicken Soup for the Woman's Soul (Chicken Soup for the Soul Series), Jack Canfield, 1998 (ISBN: 1558746226)
- The Curious Incident of the Dog in the Night-Time : A Novel, MARK HADDON, 2003 (ISBN: 0385509456)
- Middlesex: A Novel, Jeffrey Eugenides, 2002 (ISBN: 0374199698)
- Sanctuary, Nora Roberts, 2004 (ISBN: 0515122734)
- Bastard Out of Carolina, Dorothy Allison, 1993 (ISBN: 0452269571)
- The Christmas Train, David Baldacci, 2002 (ISBN: 0446525731)
- The Sweet Potato Queens' Book of Love, JILL CONNER BROWNE, 1999 (ISBN: 0609804138)
- The Dilbert Principle: A Cubicle'S-Eye View of Bosses, Meetings, Management Fads &amp; Other Workplace Afflictions, Scott Adams, 1996 (ISBN: 0887

<h1>New user: book rated before</h1>

In [37]:
"""
Fan of Sci-fi
Not a Fan of Classics

60809833 -> Brave New World, Huxley
60914653 -> The Unbearable Lightness of Being, Kundera
60934913 -> Kitchen Confidential: Adventures in the Culinary Underbelly, Anthony Bourdain
61020710 -> The Color of Magic, Terry Pratchett
64471047 -> The Lion, the Witch, and the Wardrobe (The Chronicles of Narnia, Book 2), C. S. Lewis
64471063 -> The Horse and His Boy, C. S. Lewis
64471101 -> The Magician's Nephew (rack) (Narnia), C. S. Lewis
64407667 -> The Bad Beginning (A Series of Unfortunate Events, Book 1), Lemony Snicket
64407675 -> The Reptile Room (A Series of Unfortunate Events, Book 2), Lemony Snicket
64407683 -> The Wide Window (A Series of Unfortunate Events, Book 3), Lemony Snicket
140042393 -> The Grapes of Wrath, John Steinbeck
140042520 -> Dharma Bums, Jack Kerouac
140042598 -> On the Road, Jack Kerouac
140053204 -> Travels With Charley: In Search of America, John Steinbeck
345303067 -> 2010: Odyssey Two, Arthur C. Clarke
345339703 -> The Fellowship of the Ring (The Lord of the Rings, Part 1), J.R.R. TOLKIEN
345339711 -> The Two Towers (The Lord of the Rings, Part 2), J.R.R. TOLKIEN
345389964 -> A Son of the Circus, John Irving
"""

ratings_new_userid = 123457
ratings_new_isbn = ['0060809833', 
                    '0060914653', 
                    '0060934913', 
                    '0061020710', 
                    '0064471047', 
                    '0064471063', 
                    '0064471101', 
                    '0064407667', 
                    '0064407675', 
                    '0064407683', 
                    '0140042393', 
                    '0140042520', 
                    '0140042598', 
                    '0140053204', 
                    '0345303067', 
                    '0345339703', 
                    '0345339711', 
                    '0345389964']
ratings_new_rating = [9,2,1,8,9,10,8, 7, 7, 8, 0, 3, 1, 2, 8, 9, 10, 4]
ratings_new = pd.DataFrame({
    "User-ID": ratings_new_userid,
    "ISBN": ratings_new_isbn,
    "Book-Rating": ratings_new_rating
})

In [38]:
# Features

In [39]:
check_isbns(ratings_new_isbn)

18/18 valid ISBNs.


In [40]:
# Preprocessing nových dat
ratings_new = dataset_preprocessing(ratings_new)

# Spojení v celým Ratings datasetem
ratings = pd.concat([ratings, ratings_new], ignore_index=True)

In [41]:
full_dataset.fit_partial(users=ratings_new["User-ID"].unique(), 
                         items=ratings_new["ISBN"].unique())

user_id_map, user_feature_map, item_id_map, item_feature_map = full_dataset.mapping()

user_id_inverse_map = {v: k for k, v in user_id_map.items()}
item_id_inverse_map = {v: k for k, v in item_id_map.items()}

In [42]:
new_interactions, new_weights = input_preparation(ratings_new, full_dataset)

In [43]:
# Fit Partial for Model (model.fit_partial) nefunguje pro nové uživatele - dimenzionalita nesedí

model.fit(new_interactions,
          sample_weight=new_weights,
          num_threads=4,
          verbose=True,
         )

Epoch: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 2058.05it/s]


In [44]:
user_id = 123457
no_items = 10

previous_ratings(user_id, no_items) 
print("")
prediction(user_id, PredictionType.BEST, no_items, remove_rated=True)

User ID: 123457. Previous ratings (highest to lowest):
- 10*, The Two Towers (The Lord of the Rings, Part 2), J.R.R. TOLKIEN, 1986 (ISBN: 0345339711)
- 10*, The Horse and His Boy, C. S. Lewis, 1994 (ISBN: 0064471063)
- 9*, Brave New World, Aldous Huxley, 1989 (ISBN: 0060809833)
- 9*, The Fellowship of the Ring (The Lord of the Rings, Part 1), J.R.R. TOLKIEN, 1986 (ISBN: 0345339703)
- 9*, The Lion, the Witch, and the Wardrobe (The Chronicles of Narnia, Book 2), C. S. Lewis, 1994 (ISBN: 0064471047)
- 8*, 2010: Odyssey Two, Arthur C. Clarke, 1984 (ISBN: 0345303067)
- 8*, The Wide Window (A Series of Unfortunate Events, Book 3), Lemony Snicket, 2000 (ISBN: 0064407683)
- 8*, The Magician's Nephew (rack) (Narnia), C. S. Lewis, 2002 (ISBN: 0064471101)
- 8*, The Color of Magic, Terry Pratchett, 2000 (ISBN: 0061020710)
- 7*, The Bad Beginning (A Series of Unfortunate Events, Book 1), Lemony Snicket, 1999 (ISBN: 0064407667)

Predicted: best 10:
- The Celestine Prophecy (Celestine Prophecy), Jame

<h1>New user: no ratings before (cold start)</h1>

<h1>Model save</h1>

In [46]:
with open('lightfm_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [49]:
with open('lightfm_dataset.pkl', 'wb') as dataset_file:
    pickle.dump(full_dataset, dataset_file)